<a href="https://colab.research.google.com/github/Hagar633/Machine-learning-/blob/main/symmterycalcver60.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

from collections import deque
import torch
import depth_pro
model, transform = depth_pro.create_model_and_transforms()
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device).eval()
model = model.to(device).eval()
from pickle import FRAME
from google.colab.patches import cv2_imshow
from decord import VideoReader
import mediapipe as mp
import cv2
import numpy as np
from matplotlib import pyplot as plt
import statistics
import math
import os
from torchvision import models, transforms
from PIL import Image
from sklearn.linear_model import RANSACRegressor
from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_id = "nvidia/segformer-b0-finetuned-ade-512-512"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_id)

model1 = SegformerForSemanticSegmentation.from_pretrained(model_id).to(device).eval()

preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

def approximate_intrinsics(width, height):
    c_x, c_y = width/2, height/2
    return float(c_x), float(c_y)


def symmetry_calc(left, right):
    si = abs(left - right) / (0.5 * (left + right))

    return {
        "symmetry": round(si, 3)
    }

def project_point_to_plane(point, plane):
    a, b, c, d = plane
    x0, y0, z0 = point
    normal = np.array([a, b, c], dtype=float)
    normal /= np.linalg.norm(normal)  # normalize

    # signed distance
    dist = (a*x0 + b*y0 + c*z0 + d) / np.linalg.norm([a,b,c])

    # projection onto plane
    projected = np.array([x0, y0, z0]) - dist * normal
    return projected, abs(dist)

def safe_point_to_plane_distance(point, plane_model):
    """
    Robust version of point-to-plane distance.
    Handles None planes and normalizes.
    """
    if plane_model is None:
        return None
    a, b, c, d = plane_model
    X, Y, Z = point
    denom = np.sqrt(a**2 + b**2 + c**2)
    if denom < 1e-6:
        return None
    return abs(a*X + b*Y + c*Z + d) / denom

def pixel_to_camera_coords(x, y, Z, f_x, f_y, c_x, c_y):
   X = (x - c_x) * Z / f_x
   Y = (y - c_y) * Z / f_y
   return X, Y, Z

def camera_to_pixel_coords(X, Y, Z, f_x, f_y, c_x, c_y):
    u = int((X * f_x) / Z + c_x)
    v = int((Y * f_y) / Z + c_y)
    return u, v

def point_to_plane_distance(point, plane_model):
    a, b, c, d = plane_model
    X, Y, Z = point
    return abs(a*X + b*Y + c*Z + d) / np.sqrt(a**2 + b**2 + c**2)

def median_depth_patch(depth_map, u, v, radius=3):
    """Return median depth around pixel (u,v). radius in pixels."""
    h, w = depth_map.shape
    u0 = max(0, u-radius); u1 = min(w-1, u+radius)
    v0 = max(0, v-radius); v1 = min(h-1, v+radius)
    patch = depth_map[v0:v1+1, u0:u1+1].ravel()
    patch = patch[np.isfinite(patch) & (patch > 0)]
    if len(patch)==0:
        return None
    return float(np.median(patch))

def fit_local_ground_plane_3d(depth_map, ground_mask, fx, fy, cx, cy,
                              u_center, v_center, window=80, sample_step=4, min_pts=50):
    """
    Fit plane in camera coordinates using ground_mask *near* (u_center,v_center).
    Returns plane (a,b,c,d) for ax + by + cz + d = 0 and list of sampled 3D points used.
    """
    h, w = depth_map.shape
    u0 = max(0, int(u_center - window))
    u1 = min(w-1, int(u_center + window))
    v0 = max(0, int(v_center - window))
    v1 = min(h-1, int(v_center + window))

    us, vs = np.meshgrid(np.arange(u0, u1+1, sample_step),
                         np.arange(v0, v1+1, sample_step))
    us = us.ravel(); vs = vs.ravel()

    mask_vals = ground_mask[vs, us]
    zs = depth_map[vs, us]
    keep = mask_vals & np.isfinite(zs) & (zs > 0.01)
    if keep.sum() < min_pts:
        return None, None  # not enough local ground points

    us_k = us[keep]; vs_k = vs[keep]; zs_k = zs[keep]
    # backproject to camera coords
    Xs = (us_k - cx) * zs_k / fx
    Ys = (vs_k - cy) * zs_k / fy
    Zs = zs_k
    pts3 = np.vstack([Xs, Ys, Zs]).T

    # fit Z = a*X + b*Y + c  via RANSAC (robust)
    XY = pts3[:, :2]
    Zvals = pts3[:, 2]
    ransac = RANSACRegressor(residual_threshold=0.03, min_samples=20, max_trials=200)
    ransac.fit(XY, Zvals)
    a, b = ransac.estimator_.coef_
    c = float(ransac.estimator_.intercept_)
    # plane: Z = aX + bY + c  ->  aX + bY - Z + c = 0
    # get normalized (A,B,C,D)ر
    A, B, C, D = a, b, -1.0, c
    norm = np.array([A, B, C], dtype=float)
    norm /= np.linalg.norm(norm)
    D = D / np.linalg.norm([A,B,C])
    return (norm[0], norm[1], norm[2], D), pts3

def distance_3d(p1, p2):
    return math.sqrt(
        (p1[0] - p2[0])**2 +
        (p1[1] - p2[1])**2 +
        (p1[2] - p2[2])**2
    )

def support_polygon_decision(points, plane=None):
    # points contains left_hip, right_hip, left_ankle, right_ankle

    # 1. Mid-hip point = approximate CoM
    com = (
        (points["left_hip"][0] + points["right_hip"][0]) / 2,
        (points["left_hip"][1] + points["right_hip"][1]) / 2,
        (points["left_hip"][2] + points["right_hip"][2]) / 2,
    )

    if plane is not None:
        # 2a. Project CoM vertically onto ground plane
        com_proj, _ = project_point_to_plane(com, plane)
    else:
        # 2b. If no plane: use 2D (just keep x)
        com_proj = com

    # 3. Compare horizontal distances to feet
    left_dist = np.linalg.norm(np.array(com_proj[:2]) - np.array(points["left_foot"][:2]))
    right_dist = np.linalg.norm(np.array(com_proj[:2]) - np.array(points["right_foot"][:2]))

    if left_dist < right_dist * 0.8:  # significantly closer
        return 0
    elif right_dist < left_dist * 0.8:
        return 1
    else:
        return 2

def angle_between_points(p0, p1, p2):
    """
    Calculate the angle (in radians) at p0 formed by p1 and p2.
    """
    v1 = np.array(p1) - np.array(p0)
    v2 = np.array(p2) - np.array(p0)

    dot = np.dot(v1, v2)
    norm_product = np.linalg.norm(v1) * np.linalg.norm(v2)

    if norm_product == 0:
        return None  # avoid division by zero

    cos_theta = dot / norm_product
    # Clamp value to [-1, 1] to avoid floating point errors
    cos_theta = np.clip(cos_theta, -1.0, 1.0)

    return np.degrees(np.arccos(cos_theta))


def test_l_r(i,frame,ground_mask,depth_map,fx,fy,c_x,c_y,results,width,height):
  right= False
  left= False

  plane, _ = fit_local_ground_plane_3d(depth_map, ground_mask, fx, fy, c_x, c_y,
                                     u_center=width//2, v_center=height-50,
                                     window=300, sample_step=4)
  if plane is not None:
     a, b, c, d = plane
  step = 20  # density of grid (adjust)
  for v in range(0, frame.shape[0], step):
      for u in range(0, frame.shape[1], step):
              Z = depth_map[v, u]
              if Z <= 0:
                      continue
              X, Y, Z = pixel_to_camera_coords(u, v, Z, fx, fy, c_x, c_y)

        # Check if point is near plane
              dist = abs(a * X + b * Y +c * Z + d) / np.sqrt(a**2 + b**2 + c**2)
              if dist < 0.05:  # 5cm tolerance to mark as "on plane"
                 cv2.circle(frame, (u, v), 2, (0, 255, 0), -1)
  points = {}
  mp_pose = mp.solutions.pose
  h, w, _ = frame.shape
  if results.pose_landmarks:
           for id, lm in enumerate(results.pose_landmarks.landmark):
             cx= min(max(int(lm.x * w), 0), w - 1)
             cy= min(max(int(lm.y * h), 0), h - 1)
             cz = depth_map[cy, cx]

             X, Y, Z = pixel_to_camera_coords(cx, cy, cz, fx, fy, c_x, c_y)
             if id == mp_pose.PoseLandmark.LEFT_ANKLE.value:
                points["left_ankle"] = (X, Y, Z)
             elif id == mp_pose.PoseLandmark.RIGHT_ANKLE.value:
                points["right_ankle"] = (X, Y, Z)
             elif id == mp_pose.PoseLandmark.RIGHT_KNEE.value:
                points["right_knee"] = (X, Y, Z)
             elif id == mp_pose.PoseLandmark.LEFT_KNEE.value:
                points["left_knee"] = (X, Y, Z)
             elif id == mp_pose.PoseLandmark.RIGHT_HIP.value:
                points["right_hip"] = (X, Y, Z)
             elif id == mp_pose.PoseLandmark.LEFT_HIP.value:
                points["left_hip"] = (X, Y, Z)
             elif id== mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value:
                points["left_foot"] = (X, Y, Z)
             elif id == mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value:
                points["right_foot"] = (X, Y, Z)
           disttr= distance_3d(points["right_ankle"], points["right_hip"])
           disttl= distance_3d(points["left_ankle"], points["left_hip"])
           angle_r= angle_between_points(points["right_knee"], points["right_hip"], points["right_ankle"])
           angle_l = angle_between_points(points["left_knee"], points["left_hip"], points["left_ankle"])

           if all(k in points for k in ["left_hip", "left_knee", "left_ankle"]):

                u_hip, v_hip = camera_to_pixel_coords(*points["left_hip"], fx, fy, c_x, c_y)
                u_knee, v_knee = camera_to_pixel_coords(*points["left_knee"], fx, fy, c_x, c_y)
                u_ankle, v_ankle = camera_to_pixel_coords(*points["left_ankle"], fx, fy, c_x, c_y)

                cv2.line(frame, (u_hip, v_hip), (u_knee, v_knee), (0, 255, 0), 2)
                cv2.line(frame, (u_knee, v_knee), (u_ankle, v_ankle), (0, 255, 0), 2)

           if all(k in points for k in ["right_hip", "right_knee", "right_ankle"]):
              u_hip, v_hip = camera_to_pixel_coords(*points["right_hip"], fx, fy, c_x, c_y)
              u_knee, v_knee = camera_to_pixel_coords(*points["right_knee"], fx, fy, c_x, c_y)
              u_ankle, v_ankle = camera_to_pixel_coords(*points["right_ankle"], fx, fy, c_x, c_y)


              cv2.line(frame, (u_hip, v_hip), (u_knee, v_knee), (255, 0, 0), 2)  # blue
              cv2.line(frame, (u_knee, v_knee), (u_ankle, v_ankle), (255, 0, 0), 2)


           distl = safe_point_to_plane_distance(points.get("left_ankle"), plane)
           distr = safe_point_to_plane_distance(points.get("right_ankle"), plane)
           print(f"distl: {distl}")
           print(f"distr: {distr}")

           proj_left, dist_left = project_point_to_plane(points["left_ankle"], plane)

           u1, v1 = camera_to_pixel_coords(*points["left_ankle"], fx, fy, cx, cy)
           u2, v2 = camera_to_pixel_coords(*proj_left, fx, fy, cx, cy)
           cv2.line(frame, (u1, v1), (u2, v2), (0, 255, 0), 2)

           cv2.line(frame, (u1, v1), (u2, v2), (0, 255, 0), 2)   # green line
           cv2.circle(frame, (u1, v1), 5, (0, 0, 255), -1)       # ankle (red)
           cv2.circle(frame, (u2, v2), 5, (255, 0, 0), -1)       # projection (blue)
           cv2.putText(frame, f"{dist_left:.3f} m", (u1+10, v1-10),
           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

           ankle_3d = points["right_ankle"]
           proj_right, dist_right = project_point_to_plane(ankle_3d, plane)

           u1, v1 = camera_to_pixel_coords(*ankle_3d, fx, fy, cx, cy)
           u2, v2 = camera_to_pixel_coords(*proj_right, fx, fy, cx, cy)

           cv2.line(frame, (u1, v1), (u2, v2), (255, 255, 0), 2) # cyan line
           cv2.circle(frame, (u1, v1), 5, (0, 0, 255), -1)       # ankle
           cv2.circle(frame, (u2, v2), 5, (0, 255, 255), -1)     # projection
           cv2.putText(frame, f"{dist_right:.3f} m", (u1+10, v1-10),
           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)


           left_contact  = distl  is not None and distl  < distr
           right_contact = distr  is not None and distr  < distl


           if left_contact and distr-distl >= 0.001:
              truth = "left"
              left = True

           elif right_contact and distl-distr >= 0.001:
                 truth = "right"
                 right = True

           else:
               truth = "none"


           text = f"Frame: {i} | touching: {truth}| angle_r: {angle_r:.4f}|angle_l:{angle_l:.4f}"

           font = cv2.FONT_HERSHEY_SIMPLEX
           font_scale = 2.0
           thickness = 1
           (text_w, text_h), baseline = cv2.getTextSize(text, font, font_scale, thickness)
           if text_w > frame.shape[1] - 20:  # keep margin
               font_scale = (frame.shape[1] - 10) / (0.5*text_w)
           x = 10
           y = 50 + text_h
           cv2.putText(frame, text, (x, y), font, font_scale, (0, 0, 200), thickness, cv2.LINE_AA)
           text2 = f"distl: {distl:.3f} m | distr: {distr:.3f} m"
           y2 = y + text_h + 10  # 10px spacing below first line
           cv2.putText(frame, text2, (x, y2), font, font_scale, (0, 100, 255), thickness, cv2.LINE_AA)

  return right, left


def processvideo(video_path, output_path,model):
    left=0
    right=0
    left_prev = False
    right_prev= False
    S_I=0
    vr = VideoReader(video_path)
    frame_count = len(vr)
    print(f"Total frames: {frame_count}")
    fps = 7
    step = fps * 1
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=True)
    mp_draw = mp.solutions.drawing_utils
    sample_frame = vr[0].asnumpy()
    rgb = cv2.cvtColor(sample_frame , cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)
    img1 = transform(rgb).unsqueeze(0).to(device)
    height, width, _ = sample_frame.shape
    c_x,c_y= approximate_intrinsics(width, height)
    with torch.no_grad():
            pred = model.infer(img1)
            depth_map = pred["depth"].squeeze().cpu().numpy()
            depth_map = cv2.resize(depth_map, (sample_frame.shape[1], sample_frame.shape[0]))
            intrinsics = pred["focallength_px"].cpu().numpy()
            if np.ndim(intrinsics) == 0:
              fx = fy = float(intrinsics)
            else:
              fx, fy = intrinsics
    print("kk")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for i in range(0, frame_count, step):
        frame = vr[i].asnumpy()
        rgb = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)
        results = pose.process(rgb)
        #ectracting ground
        img = transform(rgb).unsqueeze(0).to(device)
        inputs = feature_extractor(rgb, return_tensors="pt").to(device)
        with torch.no_grad():
             outputs = model1(**inputs)
             seg = outputs.logits.argmax(dim=1).squeeze().cpu().numpy()
             seg_resized = cv2.resize(seg.astype(np.uint8),
                         (depth_map.shape[1], depth_map.shape[0]),
                         interpolation=cv2.INTER_NEAREST)

        ground_ids = [3, 9, 13, 29, 94] # floor, grass, earth,field, land
        ground_mask = np.isin(seg_resized, ground_ids)
        cv2.imwrite("ground_mask.png", ground_mask.astype(np.uint8) * 255)
        right_t, left_t= test_l_r(i,frame,ground_mask,depth_map,fx,fy,c_x,c_y,results,width,height)

        if right_t and left_t== False:
          if right_prev == False:
            right_prev=  True
            left_prev= False
            right+=1
        elif left_t and right_t== False:
          if left_prev == False:
            left_prev= True
            right_prev= False
            left+=1
        else:
          right_prev= False
          left_prev= False


        out.write(frame)
    if left > 0 or right !=0:
       S_I= symmetry_calc(left,right)
       print(S_I)

    black_frame = np.zeros_like(frame)
    for _ in range(fps * 3):

         frame_with_text = black_frame.copy()

         txt = f"{S_I}"
         txt1 = f"left: {left}"
         txt2 = f"right: {right}"

         font = cv2.FONT_HERSHEY_SIMPLEX
         font_scale = 2.0
         thickness = 3
         color = (255, 255, 255)


         (text_width, text_height), baseline = cv2.getTextSize(txt, font, font_scale, thickness)
         (text_width1, text_height1), baseline = cv2.getTextSize(txt1, font, font_scale, thickness)
         (text_width2, text_height2), baseline = cv2.getTextSize(txt2, font, font_scale, thickness)


         x = (width - text_width) // 2
         y = (height + text_height) // 2
         x1 = (width - text_width1) // 2
         y1 = (height + 4* text_height1) // 2
         x2 = (width - text_width2) // 2
         y2 = (height + 8* text_height2) // 2


         cv2.putText(frame_with_text,
            txt,
            (x, y),
            font,
            font_scale,
            color,
            thickness,
            cv2.LINE_AA)
         cv2.putText(frame_with_text,
            txt1,
            (x1, y1),
            font,
            font_scale,
            color,
            thickness,
            cv2.LINE_AA)
         cv2.putText(frame_with_text,
            txt2,
            (x2, y2),
            font,
            font_scale,
            color,
            thickness,
            cv2.LINE_AA)

         out.write(frame_with_text)
    out.release()


input_folder = "/content/drive/MyDrive/test"
output_folder = "/content/drive/MyDrive/testff"
os.makedirs(output_folder, exist_ok=True)

video_files = [f for f in os.listdir(input_folder) if f.lower().endswith((".mp4", ".avi", ".mov"))]

for video in video_files:
    input_path = os.path.join(input_folder, video)
    output_name = os.path.splitext(video)[0] + "_fin.mp4"
    output_path = os.path.join(output_folder, output_name)

    print(f"Processing {video} ...")
    processvideo(input_path, output_path,model)
    print(f"Saved result to {output_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/segformer/feature_extraction_segformer.py:30: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/segformer/feature_extraction_segformer.py:35: UserWarning: The following named arguments are not valid for `SegformerFeatureExtractor.__init__` and were ignored: 'feature_extractor_type', 'reduce_labels'
  super().__init__(*args, **kwargs)


Processing kid1.mp4 ...
Total frames: 223
kk
distl: 0.09083159344003877
distr: 0.07632005595279479
distl: 0.07539587424058204
distr: 0.0007468202407168168
distl: 0.1181826402064672
distr: 0.0002908070733553814
distl: 0.0011920861527180415
distr: 0.00022937498054309557
distl: 0.07833989792605468
distr: 0.0009704274385770041
distl: 0.0658661511806885
distr: 0.00031522206102896355
distl: 0.0010932963329712833
distr: 0.00019210976153871687
distl: 0.0006449557356658088
distr: 0.0009382363888654767
distl: 0.00041471993383568334
distr: 0.0005181422110154267
distl: 0.09374882545934927
distr: 0.0014563009666217488
distl: 0.11175456810043195
distr: 0.0008757518610900129
distl: 0.0018542627951507742
distr: 0.0006473880089835627
distl: 5.259806911306342e-05
distr: 0.00033797735950902297
distl: 0.06560919416035561
distr: 0.16075568296641277
distl: 0.02664105090854463
distr: 0.15389292582728656
distl: 0.0033118871452829657
distr: 0.0001640682716340658
distl: 0.10612682376680341
distr: 0.000745188983

In [1]:
!pip install mediapipe
!pip install decord


!pip install git+https://github.com/apple/ml-depth-pro.git
!mkdir -p checkpoints
!wget https://huggingface.co/apple/DepthPro/resolve/main/depth_pro.pt -O checkpoints/depth_pro.pt

  Cloning https://github.com/apple/ml-depth-pro.git to /tmp/pip-req-build-fdexg9gi
  Running command git clone --filter=blob:none --quiet https://github.com/apple/ml-depth-pro.git /tmp/pip-req-build-fdexg9gi
  Resolved https://github.com/apple/ml-depth-pro.git to commit 9efe5c1def37a26c5367a71df664b18e1306c708
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 30.9 MB/s eta 0:00:00
  Created wheel for depth_pro: filename=depth_pro-0.1-py3-none-any.whl size=27593 sha256=da4b6edc482d64ab94766e604a9125d81c47bef56b23fc737d9548461e677d23
  Stored in directory: /tmp/pip-ephem-wheel-cache-8k008cvl/wheels/3f/ad/4e/a2b790772df19931cab8bf9c5af51665ae6502f39589e662c4
Successfully built depth_pro
--2025-09-29 17:39:22--  https://huggingface.co/apple/DepthPro/resolve/main/depth_pro.pt
Resolving huggingface.co (huggingface.co)... 54.230.71.2, 54.230.71.56, 54.230